<a href="https://colab.research.google.com/github/imrchen/sturdy-garbanzo/blob/master/Everything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore PyTorch

In [11]:
import sys
import torch, torchvision

try:
    from google.colab import drive, files
    in_colab = True
except ModuleNotFoundError:
    in_colab = False

if in_colab:
    home_dir = ''
    drive.mount('/content/drive')
    groot_dir = '/content/drive/My Drive/'
else:
    from pathlib import Path
    home_dir = str(Path.home())
    groot_dir = home_dir + '/Google Drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import *
import torch, torchvision

def display_working_env() -> None:
    print(f'This project is running on {sys.platform}')
    print(f'Python Version = {sys.version}')
    print(f'PyTorch Version = {torch.__version__}')
    print(f'TorchVision Version = {torchvision.__version__}')

local_time = lambda x, offset: x + relativedelta(hours= offset)
def local_now(hours = 8):
    return dt.datetime.now() + relativedelta(hours = hours if in_colab else 0)

def print_now():
    return print(local_now())

def DropboxLink(did, fname) -> str:
    return 'https://dl.dropboxusercontent.com/s/%s/%s' % \
    (did, fname)
    
def ungzip(original, uncompressed):
    with gzip.open(original, 'rb') as f_in:
        with open(uncompressed, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

def fetch_file_via_requests(url, save_in_dir) -> str:
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    output_fpath = save_in_dir + local_filename
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(output_fpath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return output_fpath

def start_plot(figsize=(10, 8), style = 'whitegrid', dpi = 100):
    fig = plt.figure(figsize=figsize, dpi=dpi)
    gs = fig.add_gridspec(1,1)
    plt.tight_layout()
    with sns.axes_style(style):
        ax = fig.add_subplot(gs[0,0])
    return ax

In [14]:
display_working_env()
print(f'Root Directory mounted at {groot_dir}')
print('')
print_now()

This project is running on linux
Python Version = 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
PyTorch Version = 1.6.0+cu101
TorchVision Version = 0.7.0+cu101
Root Directory mounted at /content/drive/My Drive/

2020-08-28 10:57:32.838516


# What is Tensor

In [8]:
n_row, n_col = 4, 5
x = torch.empty(n_row, n_col)
# x = torch.rand(n_row, n_col)
# x = torch.zeros(n_row, n_col, dtype=torch.int)
print(x)

tensor([[1.0474e-35, 0.0000e+00, 7.0065e-44, 6.7262e-44, 6.3058e-44],
        [6.7262e-44, 7.8473e-44, 6.3058e-44, 7.0065e-44, 7.8473e-44],
        [1.1771e-43, 6.7262e-44, 7.4269e-44, 8.1275e-44, 6.8664e-44],
        [7.1466e-44, 8.1275e-44, 6.8664e-44, 7.5670e-44, 6.4460e-44]])


In [10]:
x = x.new_ones(3, 4, dtype=torch.double, device='cuda')
print(x)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0', dtype=torch.float64)


## cuda().is_available()

- [torch.cuda](https://pytorch.org/docs/stable/cuda.html)

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    y = x.to(device)
    y = y+x
    print(y)

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]], device='cuda:0', dtype=torch.float64)


In [23]:
import torch
torch.cuda.is_available()

False

# Autograd

In [26]:
x = torch.ones(2,2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [27]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [28]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


# End of File